In [1]:
import pandas as pd

file1 = './GCJ-block-location.xlsx'

try:
    data = pd.read_excel(file1, engine='openpyxl')
except Exception as e:
    raise e

In [3]:
from coordTransform_utils import *
def parse_lat_lng(row):
    lag = row['纬度,经度'].split(',')
    ans = gcj02_to_wgs84(float(lag[0]),float(lag[1]))
    print(ans[0],ans[1])
    return ans[0],ans[1]

new_data=data.copy()
new_data[['lng','lat']] = new_data.apply(
    lambda row: parse_lat_lng(row), axis=1)

ValueError: Columns must be same length as key